In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from sklearn.cluster import KMeans
import wrangle_zillow as w

In [2]:
df = w.get_zillow_data()
df.lotsizesquarefeet

0          4506.0
1         12647.0
2          8432.0
3         13038.0
4        278581.0
           ...   
77575     47405.0
77576     12105.0
77577      5074.0
77578      6347.0
77579      5927.0
Name: lotsizesquarefeet, Length: 77580, dtype: float64

In [3]:
df= w.single_unit_properties(df)

In [4]:
df=w.handle_missing_values(df, prop_required_column = .60, prop_required_row = .60)

In [5]:
train, validate, test =w.impute_missing_values(df)

In [6]:
train, validate, test = w.impute_missing_values_1(train, validate, test)
train.head()
train.isnull().sum()

bathroomcnt                   0
bedroomcnt                    0
buildingqualitytypeid         0
sqft                          0
fips                          0
fullbathcnt                   0
latitude                      0
longitude                     0
lotsizesquarefeet             0
propertycountylandusecode     0
roomcnt                       0
unitcnt                       0
yearbuilt                     0
structuretaxvaluedollarcnt    0
home_value                    0
taxamount                     0
logerror                      0
transactiondate               0
heatingorsystemdesc           0
age                           0
taxrate                       0
acres                         0
price_per_sqft                0
lotsize_per_sqft              0
bed_bath_ratio                0
dtype: int64

In [7]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
bathroomcnt,28133.0,2.186187e+00,0.814166,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,4.500000e+00
bedroomcnt,28133.0,3.229019e+00,0.839058,1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
buildingqualitytypeid,28133.0,6.098994e+00,1.306467,1.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.200000e+01
sqft,28133.0,1.811956e+03,770.943660,3.600000e+02,1.255000e+03,1.624000e+03,2.200000e+03,6.995000e+03
fips,28133.0,6.049151e+03,21.126264,6.037000e+03,6.037000e+03,6.037000e+03,6.059000e+03,6.111000e+03
fullbathcnt,28133.0,2.124018e+00,0.790430,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
latitude,28133.0,3.402387e+07,274737.140802,3.334062e+07,3.382708e+07,3.401898e+07,3.418993e+07,3.481240e+07
longitude,28133.0,-1.181928e+08,355697.846639,-1.194753e+08,-1.183941e+08,-1.181502e+08,-1.179322e+08,-1.175593e+08
lotsizesquarefeet,28133.0,1.051998e+04,86146.692284,4.800000e+02,5.525000e+03,6.754000e+03,8.500000e+03,6.971010e+06
roomcnt,28133.0,1.898233e+00,3.120316,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.300000e+01


In [8]:
X_train, y_train, X_validate, y_validate, X_test, y_test = w.X_train_select(train, validate, test, target_var = 'logerror')

In [9]:
X_train.head()
X_train.describe()
X_train.shape


(28133, 24)

In [10]:
train.shape

(28133, 25)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28133 entries, 1616 to 32584
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   bathroomcnt                 28133 non-null  float64
 1   bedroomcnt                  28133 non-null  float64
 2   buildingqualitytypeid       28133 non-null  float64
 3   sqft                        28133 non-null  float64
 4   fips                        28133 non-null  float64
 5   fullbathcnt                 28133 non-null  float64
 6   latitude                    28133 non-null  float64
 7   longitude                   28133 non-null  float64
 8   lotsizesquarefeet           28133 non-null  float64
 9   propertycountylandusecode   28133 non-null  object 
 10  roomcnt                     28133 non-null  float64
 11  unitcnt                     28133 non-null  float64
 12  yearbuilt                   28133 non-null  float64
 13  structuretaxvaluedollarcnt  

In [12]:
train.isnull().sum()

bathroomcnt                   0
bedroomcnt                    0
buildingqualitytypeid         0
sqft                          0
fips                          0
fullbathcnt                   0
latitude                      0
longitude                     0
lotsizesquarefeet             0
propertycountylandusecode     0
roomcnt                       0
unitcnt                       0
yearbuilt                     0
structuretaxvaluedollarcnt    0
home_value                    0
taxamount                     0
logerror                      0
transactiondate               0
heatingorsystemdesc           0
age                           0
taxrate                       0
acres                         0
price_per_sqft                0
lotsize_per_sqft              0
bed_bath_ratio                0
dtype: int64

In [13]:
# Function will scale the train, validate and test
def scale_min_max():
    # create the scaler object and fit to X_train (get the min and max from X_train for each column)
    scaler = MinMaxScaler(copy=True, feature_range=(0,1)).fit(X_train)

    # transform X_train values to their scaled equivalent and create df of the scaled features
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), 
                                  columns=X_train.columns.values).set_index([X_train.index.values])
    
    # transform X_validate values to their scaled equivalent and create df of the scaled features
    X_validate_scaled = pd.DataFrame(scaler.transform(X_validate),
                                    columns=X_validate.columns.values).set_index([X_validate.index.values])

    # transform X_test values to their scaled equivalent and create df of the scaled features   
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), 
                                 columns=X_test.columns.values).set_index([X_test.index.values])
    
    return X_train_scaled, X_validate_scaled, X_test_scaled

In [14]:
X_train_scaled, X_validate_scaled, X_test_scaled= scale_min_max()

ValueError: could not convert string to float: '010M'